In [ ]:
''' Zero shot learning environment'''

In [7]:
from datasets import load_dataset

dataset = load_dataset("times_of_india_news_headlines", data_dir='./data')

Using custom data configuration default-data_dir=.%2Fdata


0 examples [00:00, ? examples/s]

Dataset times_of_india_news_headlines downloaded and prepared to /home/saket/.cache/huggingface/datasets/times_of_india_news_headlines/default-data_dir=.%2Fdata/1.1.0/089c2b11f16e5d98aed85b1188a5a0bf151170fbaaba925ec1356c4fe7b859f9. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
cnt = 0
for data in dataset['train']:
    print(data)
    
    if cnt == 100:
        break
    else:
        cnt += 1

{'publish_date': 'publish_date', 'headline_category': 'headline_category', 'headline_text': 'headline_text'}
{'publish_date': '20010101', 'headline_category': 'sports.wwe', 'headline_text': 'win over cena satisfying but defeating undertaker bigger roman reigns'}
{'publish_date': '20010102', 'headline_category': 'unknown', 'headline_text': 'Status quo will not be disturbed at Ayodhya; says Vajpayee'}
{'publish_date': '20010102', 'headline_category': 'unknown', 'headline_text': 'Fissures in Hurriyat over Pak visit'}
{'publish_date': '20010102', 'headline_category': 'unknown', 'headline_text': "America's unwanted heading for India?"}
{'publish_date': '20010102', 'headline_category': 'unknown', 'headline_text': 'For bigwigs; it is destination Goa'}
{'publish_date': '20010102', 'headline_category': 'unknown', 'headline_text': 'Extra buses to clear tourist traffic'}
{'publish_date': '20010102', 'headline_category': 'unknown', 'headline_text': 'Dilute the power of transfers; says Riberio'}
{'

In [25]:
len(dataset['train'])

3297173

In [36]:
dataset['train']

Dataset({
    features: ['publish_date', 'headline_category', 'headline_text'],
    num_rows: 3297173
})

In [37]:
# date-wise indexing
datewise_indexing = {}

for idx, date in enumerate(dataset['train']['publish_date']):
    if date not in datewise_indexing.keys():
        datewise_indexing[date] = idx

In [39]:
datewise_indexing

{'publish_date': 0,
 '20010101': 1,
 '20010102': 2,
 '20010103': 88,
 '20010104': 129,
 '20010105': 282,
 '20010106': 408,
 '20010107': 420,
 '20010108': 487,
 '20010109': 553,
 '20010110': 644,
 '20010111': 648,
 '20010123': 650,
 '20010124': 712,
 '20010125': 757,
 '20010126': 831,
 '20010127': 870,
 '20010128': 918,
 '20010129': 964,
 '20010130': 1012,
 '20010131': 1055,
 '20010201': 1094,
 '20010202': 1158,
 '20010203': 1185,
 '20010204': 1245,
 '20010205': 1278,
 '20010206': 1322,
 '20010207': 1350,
 '20010208': 1381,
 '20010209': 1415,
 '20010210': 1459,
 '20010211': 1497,
 '20010212': 1535,
 '20010213': 1559,
 '20010214': 1591,
 '20010215': 1647,
 '20010216': 1681,
 '20010217': 1717,
 '20010218': 1757,
 '20010219': 1780,
 '20010221': 1809,
 '20010222': 1850,
 '20010223': 1895,
 '20010224': 1962,
 '20010225': 2018,
 '20010226': 2044,
 '20010227': 2082,
 '20010228': 2109,
 '20010301': 2160,
 '20010302': 2194,
 '20010305': 2195,
 '20010309': 2196,
 '20010311': 2197,
 '20010313': 22

In [43]:
import pandas
from datetime import date, timedelta

In [82]:
sdate = date(2008,11,25)   # (year, month, day)
edate = date(2008,12,5) 
date_lst = list(pandas.date_range(sdate,edate+timedelta(days=1),freq='d'))
start_index = datewise_indexing[date_lst[0].strftime("%Y%m%d")]
end_index = datewise_indexing[date_lst[-1].strftime("%Y%m%d")]
start_index, end_index

(638891, 643681)

In [83]:
corpus = dataset['train']['headline_text'][start_index : end_index]

In [61]:
dataset['train']

Dataset({
    features: ['publish_date', 'headline_category', 'headline_text'],
    num_rows: 3297173
})

In [63]:
from sentence_transformers import SentenceTransformer, util


In [69]:
sbert_model = SentenceTransformer('../zero_shot_learning/pretrained_models/sbert models/all-mpnet-base-v2') 

In [85]:
corpus_embeddings = sbert_model.encode(corpus, convert_to_tensor=True)

In [90]:
queries = ['India response to terror attack']

query_embedding = sbert_model.encode(queries, convert_to_tensor=True)

In [91]:
hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=100)
hits = hits[0]      #Get the hits for the first query
for hit in hits:
    print(corpus[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))


Terror attacks: India on edge (Score: 0.8007)
Terror attack brings to fore united face of India (Score: 0.7053)
Terror struck Mumbai responds (Score: 0.7038)
Mumbai attack: Pak role under scrutiny (Score: 0.6814)
Battling jihadis: India has few options (Score: 0.6734)
What to do when terror attacks (Score: 0.6651)
We'll give a fitting reply to terrorism: Sonia (Score: 0.6620)
Watch: Mumbai terror attack (Score: 0.6580)
Watch: Mumbai terror attack (Score: 0.6580)
Terror attacks: BJP says it's time to rise above politics (Score: 0.6466)
Watch: Mumbai Terror attacks (Score: 0.6378)
India to take tips from Israel on tackling terror (Score: 0.6314)
No military action against Pakistan: India (Score: 0.6313)
Mumbai under terror attack (Score: 0.6299)
Mumbai attacks send shockwaves among Indian diaspora (Score: 0.6189)
Kolkata ill-equipped to deal with terror attacks: Cops (Score: 0.6186)
India to US: We've right to protect ourselves (Score: 0.6152)
After Mumbai attack; Centre steps up activit

In [16]:
import pandas
from datetime import date, timedelta
import json

In [13]:
class Timeline():
    
    def __init__(self):
        
        self.date_interval = {'start_date': None,
                              'end_date': None}
        self.headlines_per_point = 5
        self.topic = None
        
        # loading dataset
        self.dataset = Dataset_toi()
        self.dataset.load_dataset(dataset_dir='./data')
        
        # query based headline filter
        self.filter = Top_matches()
        
        
    def set_date_interval(self, start_date, end_date):
        '''
        set interval date for timeline creation
        args: both arguments should be 'datetime.date' object
        '''
        self.date_interval['start_date'] = start_date   # (year, month, day)
        self.date_interval['end_date'] = end_date
        
    
    def set_headlines_per_point(self, headlines_per_point):
        '''
        set value of headlines_per_point
        args: integer
        '''
        self.headlines_per_point = headlines_per_point
        
    
    def set_topic(self, topic):
        '''
        set topic on which timeline should be created
        args: topic, string
        '''
        self.topic = topic
        
    
    def timeline_points(self):
        '''
        compute points in date interval i.e. all the possible dates in the given date interval
        returns: list of Timestamp objects,
        '''
        try:
            return list(pandas.date_range(self.date_interval['start_date'],
                               self.date_interval['end_date']+timedelta(days=1),
                               freq='d'))
        except TypeError as te:
            return 0
    
    
    def headlines_grpd_by_pts(self):
        '''
        returns: headlines grouped by points, {date: headlines}
        '''
        
        # extracting all headlines point-wise
        all_headlines_pointwise = self.dataset.all_headlines_pointwise(self.timeline_points())
        
        # filtering top headlines based on topic/query
        self.filtered_headlines_pointwise = {}
        for point in all_headlines_pointwise.keys():
            self.filtered_headlines_pointwise[point] = self.filter.top_k_text(query=self.topic,
                                  texts=all_headlines_pointwise[point],
                                  k=self.headlines_per_point)
        
        return self.filtered_headlines_pointwise
    
    
    def get_timeline_json(self):
        '''
        returns: combines title and events into single json, 
                 used to create timeline in sreamlit
        '''
        
        # grouping headlines daywise
        self.headlines_grpd_by_pts()
        
        gui = GUI_streamlit()
        
        gui.create_title(self.topic, self.date_interval)
        gui.create_events(self.filtered_headlines_pointwise)
        
        
        timeline_json = gui.get_timeline_json()
        
        return json.dumps(timeline_json)

In [14]:
timeline_obj = Timeline()

Using custom data configuration default-data_dir=.%2Fdata
Reusing dataset times_of_india_news_headlines (/home/saket/.cache/huggingface/datasets/times_of_india_news_headlines/default-data_dir=.%2Fdata/1.1.0/089c2b11f16e5d98aed85b1188a5a0bf151170fbaaba925ec1356c4fe7b859f9)


  0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
start_date = date(2008, 11, 25)
end_date = date(2008, 11, 30)

timeline_obj.set_date_interval(start_date, end_date)
timeline_obj.set_topic('michael jackson death')
eval(timeline_obj.get_timeline_json())

{'title': {'text': {'headline': 'Michael Jackson Death',
   'text': 'November 25, 2008 - November 30, 2008'}},
 'events': [{'start_date': {'month': '11', 'day': '25', 'year': '2008'},
   'text': {'text': '<p>3-yr-old crushed to death</p><p>Olympian Peter Thangaraj passes away</p><p>Man drowns to death</p><p>When Man U boss Fergie cancelled dinner with Sinatra</p><p>Youth found dead; cops suspect love failure</p>'}},
  {'start_date': {'month': '11', 'day': '26', 'year': '2008'},
   'text': {'text': '<p>Killing us slowly</p><p>Heartless Kolkata watches man die</p><p>Three die in separate incidents</p><p>A month after school fight; 17-yr-old dies</p><p>Career helped Madonna get over trauma</p>'}},
  {'start_date': {'month': '11', 'day': '27', 'year': '2008'},
   'text': {'text': '<p>Top cops die on duty</p><p>Top cops die on duty</p><p>Grief therapy: Dealing with the loss</p><p>The sounds of music</p><p>Jackson may pen his memoir</p>'}},
  {'start_date': {'month': '11', 'day': '28', 'year

In [10]:
from datasets import load_dataset

In [7]:
class Dataset_toi():
    
    def __init__(self):
        self.dataset = None
        self.datewise_start_idx = None
        
    def load_dataset(self, dataset_dir):
        '''
        loads the dataset from given directory
        args: data_dir string, 
        '''
        try:
            self.dataset = load_dataset("times_of_india_news_headlines", data_dir=dataset_dir)
            
            # creating datewise start index
            self.datewise_indexing()
        except:
            return 0
        
        return 1
    
    
    def datewise_indexing(self):
        '''
        create datewise_start_idx dictionary {date: start index}
        the articles in the dataset  are sorted on date
        '''
        
        self.datewise_start_idx = {}
        for idx, date in enumerate(self.dataset['train']['publish_date']):
            if date not in self.datewise_start_idx.keys():
                self.datewise_start_idx[date] = idx
                
    
    def proc_timeline_points(self, timeline_points):
        '''
        args: timeline_points, list of timestamp objects
        returns: dict, pointwise start and end index of articles in the dataset
        '''
        pointwise_start_end_idx = {}
        
        # converting timeline_points into "yyyymmdd" format
        timeline_points = list(map(lambda x: x.strftime("%Y%m%d"), timeline_points))
        
        for idx in range(len(timeline_points) - 1):
            # checking if points exist in dataset or not 
            if timeline_points[idx] not in self.datewise_start_idx:
                continue
            if timeline_points[idx + 1] not in self.datewise_start_idx:
                continue
            
            pointwise_start_end_idx[timeline_points[idx]] = {'start_idx': self.datewise_start_idx[timeline_points[idx]],
                                                            'end_idx': self.datewise_start_idx[timeline_points[idx + 1]] - 1}
        return pointwise_start_end_idx
    
        
    def all_headlines_pointwise(self, timeline_points):
        '''
        args: timeline_points, list of timestamp objects
        returns: dict, all headlines pointwise, {date point: list of all headlines}
        '''
        # computing pontwise start and end index of articles
        pointwise_start_end_idx = self.proc_timeline_points(timeline_points)
        
        # grouping headlines point-wise
        pointwise_headlines = {}
        for point in pointwise_start_end_idx.keys():
            start_idx = pointwise_start_end_idx[point]['start_idx']
            end_idx = pointwise_start_end_idx[point]['end_idx']
            yr, mth, day = int(point[:4]), int(point[4:6]), int(point[6:])
            pointwise_headlines[date(yr, mth, day)] = self.dataset['train'][start_idx: end_idx + 1]['headline_text']
            
        return pointwise_headlines
            

In [127]:
dataset_obj = Dataset_toi()
dataset_obj.load_dataset(dataset_dir='./data')

Using custom data configuration default-data_dir=.%2Fdata
Reusing dataset times_of_india_news_headlines (/home/saket/.cache/huggingface/datasets/times_of_india_news_headlines/default-data_dir=.%2Fdata/1.1.0/089c2b11f16e5d98aed85b1188a5a0bf151170fbaaba925ec1356c4fe7b859f9)


  0%|          | 0/1 [00:00<?, ?it/s]

1

In [128]:
timeline_obj.set_date_interval(2011, 2, 22, 2011, 3, 1)

In [52]:
dataset_obj.all_headlines_pointwise(timeline_obj.timeline_points())

{'20110222': ["Need development but not at environment's cost: President",
  'Seats yes; but no sharing power; DMK to tell Congress',
  'Godhra braces for 2002 train burning verdict',
  'Boat strike at Balimela hits tribals hard',
  'Keep rage on tight leash',
  'Indians to be evacuated from Libya if situation worsens',
  'Collector is in touch with wife',
  'Libyan envoy to India quits in protest',
  'PM may announce JPC on 2G today',
  'Varanasi cleric may replace Vastanvi',
  '26/11: Bombay High Court upholds death sentence for Kasab',
  "Timothy John Roemer's American do",
  "Deepika; Salman in 'Ladies and Gentlemen'?",
  'Tanu & Manu all set to win hearts',
  'Give me any role: Manoj Bakshi',
  'Palash Sen composes a song on PM',
  'Mika clarifies all allegations',
  'Vikas Taliyan goes bald for Hair is Falling',
  "Ganesh gets his ticket to B'wood",
  'I forced myself to learn Telugu: Nithya',
  'SRK gets tattooed for Deepika',
  "Salman Khan more 'worthy' than SRK",
  'HC seeks 

In [9]:
from sentence_transformers import SentenceTransformer, util

In [26]:
class Top_matches():
    
    def __init__(self):
        self.sbert_model = SentenceTransformer('./model/pretrained_models/sbert models/all-mpnet-base-v2') 
    
    def top_k_text(self, query, texts, k):
        '''
        args: query, string, can be query, event, entity, product ...
              text, list of string, 
              k, integer, no. of headlines per point in timeline
        returns: top_k, list of string, top k matches with query
        '''
        # computing queries embedding
        query_embedding = self.sbert_model.encode(query, convert_to_tensor=True)
        
        # computing texts embedding
        texts_embedding = self.sbert_model.encode(texts, convert_to_tensor=True)
        
        # finding top k matches
        matches_id = util.semantic_search(query_embedding, texts_embedding, top_k=k)
        
        matches = []
        for match in matches_id[0]:
            matches.append([texts[match['corpus_id']], match['score']])
            
        return matches

In [27]:
matches_obj = Top_matches()

In [71]:
matches_obj.top_k_text(query = "terror attack",
                      texts = ["Need development but not at environment's cost: President",
  'Seats yes; but no sharing power; DMK to tell Congress',
  'Godhra braces for 2002 train burning verdict',
  'Boat strike at Balimela hits tribals hard',
  'Keep rage on tight leash',
  'Indians to be evacuated from Libya if situation worsens',
  'Collector is in touch with wife',
  'Libyan envoy to India quits in protest',
  'PM may announce JPC on 2G today',
  'Varanasi cleric may replace Vastanvi',
  '26/11: Bombay High Court upholds death sentence for Kasab',
  "Timothy John Roemer's American do",
  "Deepika; Salman in 'Ladies and Gentlemen'?",
  'Tanu & Manu all set to win hearts',
  'Give me any role: Manoj Bakshi',
  'Palash Sen composes a song on PM',
  'Mika clarifies all allegations',
  'Vikas Taliyan goes bald for Hair is Falling',
  "Ganesh gets his ticket to B'wood",
  'I forced myself to learn Telugu: Nithya',
  'SRK gets tattooed for Deepika',
  "Salman Khan more 'worthy' than SRK",
  'HC seeks info on action against erring builders; architects',
  'Raja; Kanimozhi skip House address',
  "In IAF; sacking is a lesser punishment than 3 months' jail: SC",
  'Khaps have to pick up Rs33cr tab for loss to Rlys during stir: SC',
  'Decision on cancer drug for infertility cure today',
  'Left scuppered Sonia chance of becoming PM in 1999',
  'Vastnavi may lose Deoband post as rivals band together',
  "School; govt get HC notice over 'denial' of admission",
  'On March 1; poll dates to be out for Bengal; TN & 3 others',
  "Congress Telangana MPs disrupt Prez's address",
  'All Harsiddhi allottees to be heard in Navy case',
  'HC wants punishment for AIIMS doctors who struck work',
  'HC on Jangpura mosque',
  "Aarushi's parents move HC against CBI court order",
  'Doctors say they may appeal against order',], k = 5)

[['Godhra braces for 2002 train burning verdict', 0.29491132497787476],
 ['HC on Jangpura mosque', 0.22771818935871124],
 ['Indians to be evacuated from Libya if situation worsens',
  0.21483074128627777],
 ["Congress Telangana MPs disrupt Prez's address", 0.20602723956108093],
 ['HC wants punishment for AIIMS doctors who struck work',
  0.20456922054290771]]

In [12]:
# creating timeline json
class GUI_streamlit():
    
    def __init__(self):
        self.title = None
        self.events = None
        self.timeline_json = None
        
        
    def create_title(self, topic, date_interval):
        '''
        args: topic, str, title of the timeline
              date_interval, dict, time interval of the timeline
        '''
        timeline_start_dt = timeline_obj.date_interval['start_date'].strftime('%B %d, %Y')
        timeline_end_dt = timeline_obj.date_interval['end_date'].strftime('%B %d, %Y')
        timeline_range_display = timeline_start_dt + ' - ' + timeline_end_dt
        self.title = {
            "text": {
                "headline": topic.title(),
                "text": timeline_range_display
            }
        }
    
    
    def create_events(self, daywise_events):
        '''
        args: daywise_events, dict, {date: list of headlines}
        creating events for streamlit timeline
        '''
        
        self.events = []
        for point in daywise_events.keys():
            
            # creating start_date of an event
            day = point.strftime('%d')
            month = point.strftime('%m')
            year = point.strftime('%Y')
            
            # creating text of an event
            event_text = ''
            for headline in daywise_events[point]:
                event_text = event_text + '<p>' + headline[0] + '</p>'
            
            # creating an event
            event = {
                "start_date": {
                    "month": month,
                    "day": day,
                    "year": year
                },
                "text": {
                    "text": event_text
                }
            }
            
            # appending an event in the event list
            self.events.append(event)
            
        return self.events
    
    
    def get_timeline_json(self):
        '''
        returns: combines title and events into single json
        '''
        
        self.timeline_json = {
            "title": self.title,
            "events": self.events
        }
        
        return self.timeline_json

In [94]:
timeline_obj.headlines_grpd_by_pts()

{datetime.date(2011, 2, 22): [['Wedding procession in Kadi attacked',
   0.42795073986053467],
  ["'Wrong' riot victims: HC notice to administration", 0.4249263405799866],
  ['Police raid and seal Bagha Jatin club', 0.39109233021736145],
  ["'Wrong' riot victims: HC notice to admn", 0.3768361806869507],
  ["Book 22 cops for kidnap & murder: Ishrat's mother", 0.34571200609207153]],
 datetime.date(2011, 2, 23): [["'K'taka minorities attack is state sponsored terror'",
   0.5516824722290039],
  ['NDFB reprisal threat against anti-terror ops', 0.4903494715690613],
  ['Savage attacks on girls on a rise', 0.43862777948379517],
  ['Painful memories for Hondh-Chillar riot victims', 0.3949401378631592],
  ["Abu Salem hit out at Dossa in self-defence: DCP's affidavit",
   0.36383694410324097]],
 datetime.date(2011, 2, 24): [['Cos seek urgent evacuation',
   0.3797425925731659],
  ['Turmoil; calamities may hit tourism', 0.3752124607563019],
  ["Tension in Murbad over 'honour killing'", 0.35390618

In [107]:
timeline_obj.date_interval['start_date'].strftime('%B %d, %Y') + ''

'February 22, 2011'

In [1]:
import json
import re


class LazyDecoder(json.JSONDecoder):
    def decode(self, s, **kwargs):
        regex_replacements = [
            (re.compile(r'([^\\])\\([^\\])'), r'\1\\\\\2'),
            (re.compile(r',(\s*])'), r'\1'),
        ]
        for regex, replacement in regex_replacements:
            s = regex.sub(replacement, s)
        return super().decode(s, **kwargs)

In [3]:
# load data
with open('./data/example.json', "r") as f:
    data = json.load(f, cls=LazyDecoder)


In [ ]:
hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=100)
hits = hits[0]      #Get the hits for the first query
for hit in hits:
    print(corpus[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))


In [ ]:
# date-wise indexing
datewise_indexing = {}

for idx, date in enumerate(dataset['train']['publish_date']):
    if date not in datewise_indexing.keys():
        datewise_indexing[date] = idx

In [ ]:
from datasets import load_dataset

dataset = load_dataset("times_of_india_news_headlines", data_dir='./data')